# Interactive Sentinel-2 scene classification


### Initialize Google Earth Engine and geemap libraries

In [1]:
# Import libraries and authenticate with Google Earth Engine (run every time you start the notebook.)
import os
import ee
import geemap

May need to update the geemap package (Uncomment and run only once. Then comment out this line before re-running.)

In [ ]:
# geemap.update_package()

In [2]:
# Initialize earth engine library (run every time you start the notebook.)
ee.Initialize()

The following may not be needed. An authentication call is built into geemap library, but if it does not work, run this cell.

In [ ]:
# Authenticate to Google Earth Engine Servers.
ee.Authenticate()

### Define study area and period

Multiple options for defining a region of interest (roi) are shown below, including by latitude and longitude or by country boundary, or by a group of countries. Define only one roi by commenting out the rest.

In [3]:
# Define region of interest by a point (latitude and longitude) with a buffer.
buffer = 5000     # enter a radius in meters
lon = 7.273971    # e.g., Ryggfonn is at lon = 7.273971 and lat = 61.964367
lat = 61.964367 
roi = ee.Geometry.Point(lon, lat).buffer(buffer) 

In [ ]:
# Define region of interest by a country boundary.
#roi = ee.FeatureCollection('USDOS/LSIB/2017').filter(ee.Filter.eq('COUNTRY_NA','Norway'))

In [ ]:
# Define region of interest by a collection of countries. Note, this runs slowly. To run, uncomment all following lines.
#countries = ee.FeatureCollection([
#    ee.Feature(ee.FeatureCollection('USDOS/LSIB/2017').filter(ee.Filter.eq('COUNTRY_NA', 'Norway'))),
#    ee.Feature(ee.FeatureCollection('USDOS/LSIB/2017').filter(ee.Filter.eq('COUNTRY_NA', 'Sweden'))),
#    ee.Feature(ee.FeatureCollection('USDOS/LSIB/2017').filter(ee.Filter.eq('COUNTRY_NA', 'Finland')))
#])
#region = countries.flatten()                                                     
#roi = region.geometry()

When selecting the time period of interest, consider that a mosaic image will be created from all images collected during that time period. When a time period is long enough to include multiple images at a location at different times, the most recent image is used. There are methods to change this default but these are not implemented here. [More reading]( https://developers.google.com/earth-engine/guides/ic_composite_mosaic)

In [4]:
# Define time period of study in the format of 'yyyy-mm-dd'
start_date = '2022-03-23'
end_date = '2022-04-23'

### Import and filter dataset

In [5]:
# Define cloud mask function per Earth Engine Catalog.
def mask_S2_clouds(image):
    qa = image.select('QA60')
    
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloud_bit_mask = 1<<10
    cirrus_bit_mask = 1<<11
    
    # Both flags should be set to zero, indicating clear conditions.
    mask = (qa.bitwiseAnd(cloud_bit_mask)
            .eq(0)
            .And(qa.bitwiseAnd(cirrus_bit_mask)
            .eq(0)))
    
    return image.updateMask(mask).divide(10000)

Import the Sentinel-2 Surface Reflectance product. Filter it to our site and to get less cloudy tiles, and mask remaining cloudy pixels.

In [6]:
# Tiles with Cloudy_Pixel_Percentage greater than cloud_threshold will be excluded from filtered collection.
cloud_threshold = 20 # Enter value in percent (0 to 100).

# Import and filter image collection.
dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate(start_date, end_date) 
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',cloud_threshold))
    .map(mask_S2_clouds)
    .filterBounds(roi)
)

In [7]:
# Confirm the number of images in the filtered collection by printing the size of dataset.
dataset.size().getInfo()

4

### Create an image from filtered collection and visualize it on the map

In [8]:
# Combine the images in the filtered collection into one image.
test_img = dataset.mosaic().clip(roi)


In [9]:
# Define visualization parameters for optical imagery.
vis_params = {
    'min': 0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'], # B4, B3, B2 are red, green, and blue bands for natural color images.
}

vis_params_snow = {
    'min': 0,
    'max': 0.3,
    'bands': ['B2', 'B11', 'B12'], # B2, B11, and B12 bands are blue, SWIR1, and SWIR2 and differentiate snow from clouds.
}

In [12]:
# Create interactive map centered at region of interest (roi).
zoom = 11 # integer represents zoom level. Lower value = zoom out, larger value = zoom in. Typical values 3 > zoom > 12.
Map = geemap.Map()
Map.addLayer(test_img, vis_params, 'RGB')
Map.addLayer(test_img, vis_params_snow, 'False color to highlight snow, clouds, and terrain')
Map.centerObject(roi, zoom)
Map

Map(center=[61.964372129441934, 7.273971302638792], controls=(WidgetControl(options=['position', 'transparent_…

### Collect training data for landcover classification

Create training data using interactive tool "Collect Training Sample" following the procedure shown [here](https://www.youtube.com/watch?v=VWh5PxXPZw0). Note in the video when the demonstrator clicks Apply. Mark enough points to define each class, which will depend on how many classes need to be defined, how unique they are from each other, and how accurate the classifier should be. Then run the following cell to collect the points into a feature collection. Scan through the output text to ensure that data points were collected for each class.

In [13]:
# Create a variable to collect these geometries as feature collection.
feature_collection = Map.user_rois.getInfo()
feature_collection

{'type': 'FeatureCollection',
 'columns': {'color': 'String',
  'label': 'String',
  'landcover': 'Integer',
  'system:index': 'String'},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [7.237815, 61.9399]},
   'id': '0',
   'properties': {'color': '#3388ff', 'label': 'water', 'landcover': 0}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [7.241549, 61.938608]},
   'id': '1',
   'properties': {'color': '#3388ff', 'label': 'water', 'landcover': 0}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [7.240949, 61.943857]},
   'id': '2',
   'properties': {'color': '#3388ff', 'label': 'water', 'landcover': 0}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [7.229955, 61.944159]},
   'id': '3',
   'properties': {'color': '#ff3333', 'label': 'soil', 'landcover': 1}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [7.229204, 61.943291]},
   'id': '4',
   'properties': 

### Sample the imagery at the training points to create training dataset and train the classifier

At each point on the map, a lot of information is available. Some information is more useful than others for separability of classes. For example, a cloud and  a patch snow look similar in the RGB image (booh white) but different in the false color image (snow is read and clouds are white). We want to train the classifier to using the data that will help it recognize our classes of interest. 

In [14]:
# Select the bands of interest for training.
bands = ['B2', 'B11', 'B12'] # blue, SWIR1, and SWIR2 bands good for distinguishing between clouds and snow.

# Sample the inmage to get a FeatureCollection of training data.
training = (test_img.select(bands)
            .sampleRegions(
               **{
                'collection': feature_collection,
                'properties': ['landcover'],
                'scale': 10}))

Train the classifier. Here, we are using a Random Forest classifier, one of several machine learning algorithms available in GEE for supervised classification. [Read more here](https://developers.google.com/earth-engine/guides/classification)

In [15]:
# Make a Random Forest classifier and train it. The user can change the number of trees used in the classifier if desired.
num_trees = 100
classifierRF = (ee.Classifier.smileRandomForest(num_trees)
                .train(
                    **{
                    'features': training,
                    'classProperty': 'landcover',
                    'inputProperties': bands}))

### Classify the image and view the results

In [16]:
# Classify the image we created above.
img_RF = test_img.select(bands).classify(classifierRF)

In [17]:
# Define a palette to visualize the different landcover classes. There must be one color per class.
palette = [
  '334a8f', # water (0) // blue
  '876e46', # soil (1)  // brown
  'f7f8fa'  # snow (2)  // off white
]

# User must enter a value equal to the number of classes, less one. E.g., if there are three classes, then max = 2.
max = 2 

# Add the classified image to a new layer and show it on the map 
Map.addLayer(img_RF, {'min': 0, 'max': max , 'palette': palette}, 'Landcover')
Map

Map(bottom=4685872.0, center=[61.941493743397906, 7.228979289945926], controls=(WidgetControl(options=['positi…

### Validate classifier

Not implemented. Classifier should be 'ground truthed' prior to operational use. Accuracy assessment could follow these [guidelines.](https://developers.google.com/earth-engine/guides/classification#accuracy-assessment)

In [ ]:
# Insert validation procedures.

### Use trained classifier to classify other imagery

The following section defines a function to classify additional imagery at the same site from different time periods, using our trained and validated classifier. It adds each new classified image as a layer to the map.

In [18]:
# Define a function to create and classify new images of the study site.
def classify_img(start, end):
    collection = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate(start, end) 
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',cloud_threshold))
    .map(mask_S2_clouds)
    .filterBounds(roi)
    )
    img_merged = collection.mosaic().clip(roi)
    img_RF = img_merged.select(bands).classify(classifierRF)
    return img_RF

In [20]:
# Create new classified images with unique dates and names.
start_1 = '2022-04-23'
end_1 = '2022-05-23'
img_1 = classify_img(start_1, end_1)

start_2 = '2022-05-23'
end_2 ='2022-06-23'
img_2 = classify_img(start_2, end_2)

In [22]:
# Add the new images to the map.
Map.addLayer(img_1, {'min': 0, 'max': max , 'palette': palette}, 'Image 1 landcover')
Map.addLayer(img_2, {'min': 0, 'max': max , 'palette': palette}, 'Image 2 landcover')

# Center map at roi and call the map to visualize.
Map.centerObject(roi, zoom)
Map

Map(bottom=293150.0, center=[61.964372129441934, 7.273971302638792], controls=(WidgetControl(options=['positio…